In [66]:
import sys
import pandas as pd
import numpy as np

# INPUT FILE
scen_file = 'esg_시나리오.xlsx'
in_file = '지수산출기초데이터.xlsx'
out_file_1 = 'convert_by_isu_'+ in_file   # xlsx로만 저장되네...
out_file_2 = 'convert_by_val_'+ in_file   # xlsx로만 저장되네...

# 개편일정보 가져오기
df = pd.read_excel(scen_file,dtype='unicode')
df = df['정기개편일']
df.drop_duplicates(inplace=True)
phase = df.values.tolist()

# input file 을 읽는다
#df = pd.read_csv(in_file, sep=',',dtype='unicode', parse_dates=True)
df = pd.read_excel(in_file, sep=',',dtype='unicode')

#숫자로 봐야하는 컬럼의 dtype을 변경한다
cols=df.columns.drop(['DATE','CODE'])
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# 필요없는 컬럼 지우기
#df.drop([df.columns[1],df.columns[2],df.columns[4],df.columns[7],df.columns[10],df.columns[11]],axis=1,inplace=True)

#
df.set_index('DATE', inplace=True)
df.sort_index(axis = 0, level=0, inplace=True, ascending=True)

# START - 1. 종목별 세부 정보 만들기. MULTI INDEX level 0 => 종목, level 1 => 세부정보
# 각 개편일별로 끊어서 list에 추가함
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # TABLE RESHAPE - 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['CODE'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    # - 종목코드를 열 index level 0로
    df2.columns = df2.columns.swaplevel(0,1)
    # - 열 index level 1(종목별 항목들)을 sorting
    df2.sort_index(axis = 1, level=0, inplace=True)
    df2.apply(pd.to_numeric, errors='ignore')
    df_list.append(df2)

# 엑셀파일저장
with pd.ExcelWriter(out_file_1) as writer:
    for item in df_list :
        item.to_excel(writer, sheet_name=item.index[0])

print ('** FILE CREATED : ' + out_file_1)
# END - 1. 종목별 세부 정보 만들기. MULTI INDEX level 0 => 종목, level 1 => 세부정보

# START - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목
# 각 개편일별로 끊어서 list에 추가함
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # TABLE RESHAPE - 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['CODE'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    df2['BACE PRICE'].apply(pd.to_numeric, errors='ignore')
    df_list.append(df2)
# 엑셀파일저장
with pd.ExcelWriter(out_file_2) as writer:
    for item in df_list :
        item.to_excel(writer, sheet_name=item.index[0])
        
print ('** FILE CREATED : ' + out_file_2)
# END - 2. 세부정보별 종목정보 만들기. MULTI INDEX level 0 => 세부정보, level 1 => 종목

#완료
print ('converting 작업을 완료하였습니다.')



** FILE CREATED : convert_by_isu_지수산출기초데이터.xlsx
** FILE CREATED : convert_by_val_지수산출기초데이터.xlsx
converting 작업을 완료하였습니다.


In [11]:
my_df = df_list[0]
my_df.columns.levels[0]

Index(['BAS_PRICE', 'CLOSE_PRICE', 'IDX_LISTED_STOCKS', 'COMP_RT', 'STYLE_RT'], dtype='object')

In [ ]:
df_list = []
for i in range(len(phase)) :
    if i == len(phase) -1 :
        df2 = df[phase[i]:]
        
    else:
        df2 = df[phase[i]:phase[i+1]]
        df2 = df2.drop(phase[i+1])
    # 종목코드 열로 옮기는 작업
    #  - 키 추가
    df2.set_index(['ISU_SRT_CD'], append=True, inplace=True) #index 추가시 append param 사용함
    # - 열로 옮김
    df2 = df2.unstack(level=1)
    # - 종목코드를 열 index level 0로
    df2.columns = df2.columns.swaplevel(0,1)
    # - 열 index level 1(종목별 항목들)을 sorting
    df2.sort_index(axis = 1, level=0, inplace=True)
    df_list.append(df2)
        


In [ ]:
df_list[1].index[0]

In [ ]:
scen_file = 'esg_시나리오.xlsx'
df3 = pd.read_excel(scen_file,dtype='unicode')

In [ ]:
#df3.set_index('정기개편일', inplace=True)
df3 = df3['정기개편일']

In [ ]:
df3.drop_duplicates(inplace=True)

In [ ]:
price = mydf['CLOSE_PRICE'].astype('int64')

In [ ]:
mydf = df_list[0]["20161004" : "20161230"]
mydf

In [ ]:
shrs = mydf['IDX_LISTED_STOCKS'].astype('int64')

In [ ]:
iif = mydf['STYLE_RT'].astype(float)

In [ ]:
portion = price*shrs*iif

In [ ]:
sum = portion.sum(axis=1,skipna = True)

In [ ]:
portion.to_excel('portion.xlsx')

In [ ]:
cap = price.mul(shrs)

In [ ]:
portion = cap.mul(iif)

In [ ]:
portion.to_excel('portion.xlsx')

In [42]:
import sys
import pandas as pd
import numpy as np

infile = "calcmaster.xls"
df = pd.read_excel(infile, dtype='unicode')

In [45]:
df2 = df[['TRADE_DATE','ISU_SRT_CD','COMP_RT']]
df2['COMP_RT'] = df2['COMP_RT'].astype(float)

C:\Users\DB400\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
df2

,TRADE_DATE,ISU_SRT_CD,COMP_RT
0,20160104,006260,0.080289
1,20160104,009830,0.080581
2,20160104,010060,0.079906
3,20160104,011930,0.055097
4,20160104,012330,0.079592
...,...,...,...
206,20201217,297090,0.010328
207,20201217,298040,0.019129
208,20201217,298050,0.026415
209,20201217,306200,0.007207


In [47]:
df3 = df2.pivot(index='TRADE_DATE',columns='ISU_SRT_CD',values='COMP_RT')

In [49]:
df3

ISU_SRT_CD,006260,009830,010060,011930,012330,018000,034020,034730,044490,059090,...,112610,126340,161000,210980,288620,297090,298040,298050,306200,336260
TRADE_DATE,,,,,,,,,,,,,,,,,,,,,
20160104,0.080289,0.080581,0.079906,0.055097,0.079592,0.031917,0.077232,0.079973,0.077551,0.027275,...,0.079786,NaN,0.056703,0.080278,NaN,NaN,NaN,NaN,NaN,NaN
20160616,0.084290,0.082144,0.077909,0.058506,0.085945,0.029087,0.061502,0.080267,0.068529,0.026895,...,0.066357,NaN,0.067697,0.088561,NaN,NaN,NaN,NaN,NaN,NaN
20161215,0.080193,0.085913,0.081938,0.052829,0.082872,0.027343,0.077713,0.076803,0.078548,0.025908,...,0.049890,NaN,0.080236,0.078859,NaN,NaN,NaN,NaN,NaN,NaN
20170615,0.080702,0.076898,0.080037,0.063462,0.077257,0.039259,0.084621,0.079921,0.078268,0.023816,...,0.049168,NaN,0.087814,0.071511,NaN,NaN,NaN,NaN,NaN,NaN
20171221,0.084601,0.082042,0.085571,0.060605,0.076754,0.068864,0.077430,0.077104,0.050423,0.029592,...,0.050006,NaN,0.081319,0.068078,NaN,NaN,NaN,NaN,NaN,NaN
20180621,0.080707,0.075425,0.065303,0.053341,0.088420,0.067130,0.077164,0.083361,0.050916,0.036269,...,0.055190,NaN,0.066474,0.074441,NaN,NaN,NaN,NaN,NaN,NaN
20181220,0.074794,0.084185,0.083142,0.026782,0.088319,0.032377,0.078255,0.074515,0.024333,0.027615,...,0.043683,NaN,0.035688,0.063159,0.014221,NaN,0.062287,0.071479,0.020783,NaN
20190620,0.079431,0.085115,0.080226,0.032678,0.078226,0.024057,0.089739,0.077600,0.023826,0.036830,...,0.062223,NaN,0.033086,0.046605,0.010150,NaN,0.046182,0.070806,0.023458,NaN
20191219,0.082654,0.082471,0.079864,0.026432,0.084976,0.022012,0.078117,0.077232,0.022982,0.038648,...,0.068776,NaN,0.039573,NaN,0.010603,0.011414,0.032122,0.078312,NaN,0.044969


In [51]:
df3 = df3.fillna(0)
df3.to_excel("weight.xlsx")

In [33]:
df2 = df[['TRADE_DATE','ISU_SRT_CD','COMP_RT']]

In [35]:
df3 = df2.pivot(index='TRADE_DATE', columns='ISU_SRT_CD',values='COMP_RT')

In [36]:
df3

ISU_SRT_CD,006260,009830,010060,011930,012330,018000,034020,034730,044490,059090,...,112610,126340,161000,210980,288620,297090,298040,298050,306200,336260
TRADE_DATE,,,,,,,,,,,,,,,,,,,,,
20160104,0.0802893332216639,0.0805809686302536,0.07990646989549299,0.055097101718418,0.0795919005304323,0.0319171216035209,0.0772322955126717,0.0799725443195592,0.0775506971731277,0.0272748157123416,...,0.0797860273943184,NaN,0.0567030866845482,0.0802784897031377,NaN,NaN,NaN,NaN,NaN,NaN
20160616,0.0842900427781094,0.0821435070432128,0.0779087029315878,0.0585059105634443,0.0859450683227386,0.0290866755898269,0.0615021102085962,0.0802666245817162,0.0685289884126221,0.0268950539329915,...,0.066357348533151,NaN,0.0676973538445587,0.0885612716760036,NaN,NaN,NaN,NaN,NaN,NaN
20161215,0.0801934675076866,0.085912930469863,0.0819376602714092,0.0528294453288134,0.0828720877483894,0.027342987888513,0.0777133033831549,0.0768034658527308,0.0785476128607255,0.025907989254692,...,0.0498904886003729,NaN,0.0802356482762279,0.078859023327551,NaN,NaN,NaN,NaN,NaN,NaN
20170615,0.0807021600092553,0.0768975312783756,0.0800369655149855,0.0634615426033044,0.0772571809013304,0.039258984960805,0.0846213743140986,0.0799209777827326,0.0782684906344639,0.0238158132331835,...,0.0491676467238226,NaN,0.0878141668841842,0.0715110094856251,NaN,NaN,NaN,NaN,NaN,NaN
20171221,0.0846011139656135,0.0820423817626252,0.08557113039641,0.0606046050380822,0.0767539357036795,0.068864069296154,0.0774303601405322,0.0771039354398538,0.0504229892873337,0.0295921015622152,...,0.0500056740564228,NaN,0.0813192168164709,0.0680782838791869,NaN,NaN,NaN,NaN,NaN,NaN
20180621,0.0807070813597846,0.0754251811466786,0.0653030206051109,0.0533409728072908,0.0884197570399153,0.06712957195466,0.0771637513757818,0.0833607892487535,0.0509160455541883,0.0362690509714793,...,0.0551903861304353,NaN,0.0664744438267891,0.074441009840945,NaN,NaN,NaN,NaN,NaN,NaN
20181220,0.074793547048465,0.0841854128969647,0.0831420787059844,0.0267816204185257,0.0883189150780141,0.0323766915788124,0.078254972542536,0.0745152535090405,0.0243331292638654,0.0276149895689948,...,0.043682546169452,NaN,0.0356875982547767,0.0631591649520371,0.0142205404715786,NaN,0.0622873816821449,0.0714787377532532,0.0207832751649204,NaN
20190620,0.0794308358813921,0.0851145208910349,0.0802255705090859,0.0326781573074567,0.0782258474691671,0.0240565341188491,0.0897387468915663,0.077599697956606,0.0238264372328113,0.0368302690152727,...,0.0622226671470229,NaN,0.0330857813766229,0.046604670186155,0.0101499855158387,NaN,0.04618194303579,0.0708061465171644,0.0234581308915117,NaN
20191219,0.0826543285477734,0.0824705738262757,0.0798636371577663,0.0264323526115095,0.0849758731572307,0.0220115638687817,0.0781169208180451,0.0772320815724415,0.022982395643023,0.0386481659857289,...,0.0687757760750492,NaN,0.0395732097017922,NaN,0.0106026481912268,0.0114141876313376,0.0321219838648974,0.078312090989999,NaN,0.0449687061443373


In [38]:
df3 = df3.fillna(0)

In [40]:
df3.to_excel("weight.xlsx")